# 🧠 MODEL EFFICIENTNET-B0 UNTUK KLASIFIKASI AFLATOKSIN

## Mengapa EfficientNet-B0?

| Aspek | ResNet50 | EfficientNet-B0 | Keuntungan |
|-------|----------|-----------------|------------|
| **Parameter** | 25.6 juta | 5.3 juta | 5x lebih kecil → kurang overfitting |
| **Top-1 ImageNet** | 76.0% | 77.1% | Lebih akurat dengan parameter lebih sedikit |
| **FLOPS** | 4.1B | 0.39B | 10x lebih efisien |

## Perubahan Utama dari ResNet50:

1. **Arsitektur dasar**: `ResNet50` → `EfficientNetB0`
2. **Classifier head lebih sederhana**: `[512, 256, 64]` → `[256]` (1 layer saja)
3. **Fine-tune layer**: Layer 140+ → Layer ~200+ (lebih sedikit layer di-unfreeze)
4. **Learning rate lebih konservatif**: Karena model lebih kecil

---

In [ ]:
# =============================================================================
# CELL 1: LOAD PREPROCESSING
# =============================================================================
# Menjalankan file preprocess_efficientnet.ipynb untuk memuat dataset
# Pastikan file tersebut ada di folder yang sama!
# =============================================================================

%run preprocess.ipynb

In [ ]:
# =============================================================================
# CELL 2: IMPORT LIBRARIES UNTUK MODEL
# =============================================================================

import os
import numpy as np
import pandas as pd
import tensorflow as tf
from datetime import datetime

# Model imports
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import (
    Dense, 
    GlobalAveragePooling2D, 
    Dropout, 
    Input,
    BatchNormalization  # Opsional, untuk stabilitas training
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, AdamW, RMSprop, SGD
from tensorflow.keras.callbacks import (
    EarlyStopping, 
    ReduceLROnPlateau, 
    ModelCheckpoint
)

# Metrics imports
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    accuracy_score
)

print("Libraries imported successfully!")

In [ ]:
# =============================================================================
# CELL 3: FOCAL LOSS (OPSIONAL - UNTUK IMBALANCED DATA)
# =============================================================================
#
# PENJELASAN FOCAL LOSS:
# 
# Standard Cross-Entropy:
#   CE(p) = -log(p)  dimana p adalah probabilitas kelas yang benar
#
# Focal Loss (Lin et al., 2017):
#   FL(p) = -α * (1-p)^γ * log(p)
#
# Parameter:
#   - α (alpha): Bobot untuk menyeimbangkan kelas (biasanya 1.0)
#   - γ (gamma): "Focusing parameter" (biasanya 2.0)
#     - gamma=0: sama dengan cross-entropy
#     - gamma>0: mengurangi loss untuk sampel yang mudah (high p)
#                sehingga model fokus pada sampel yang sulit (low p)
#
# KAPAN GUNAKAN FOCAL LOSS:
#   - Dataset imbalanced (seperti kasus Anda)
#   - Model terlalu "confident" pada kelas mayoritas
#
# KAPAN GUNAKAN CATEGORICAL CROSSENTROPY:
#   - Dataset balanced
#   - Sebagai baseline untuk perbandingan
# =============================================================================

class FocalLoss(tf.keras.losses.Loss):
    """
    Focal Loss untuk menangani class imbalance.
    Referensi: Lin et al., "Focal Loss for Dense Object Detection", ICCV 2017
    """
    
    def __init__(self, alpha=1.0, gamma=2.0, name='focal_loss'):
        """
        Args:
            alpha: Bobot balancing (default 1.0)
            gamma: Focusing parameter (default 2.0)
                   - Semakin besar gamma, semakin fokus pada sampel sulit
        """
        super().__init__(name=name)
        self.alpha = alpha
        self.gamma = gamma
    
    def call(self, y_true, y_pred):
        # Clip prediksi untuk mencegah log(0)
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1.0 - epsilon)
        
        # Hitung cross entropy
        ce = -y_true * tf.math.log(y_pred)
        
        # Hitung focal weight: (1 - p)^gamma
        # p adalah probabilitas untuk kelas yang benar
        pt = y_true * y_pred  # Ambil probabilitas kelas benar
        pt = tf.reduce_sum(pt, axis=-1, keepdims=True)
        focal_weight = tf.pow(1.0 - pt, self.gamma)
        
        # Gabungkan: α * (1-p)^γ * CE
        focal_loss = self.alpha * focal_weight * tf.reduce_sum(ce, axis=-1, keepdims=True)
        
        return tf.reduce_mean(focal_loss)

print("FocalLoss class defined.")
print("  - Default alpha=1.0, gamma=2.0")
print("  - Gunakan untuk dataset imbalanced")

In [ ]:
# =============================================================================
# CELL 4: KONFIGURASI EKSPERIMEN
# =============================================================================

# Layer Dense setelah GlobalAveragePooling
DENSE_UNITS = 256
DROPOUT_RATE = 0.5 # Semakin tinggi = regularisasi lebih kuat

# Fine-tune dari layer ke berapa? EfficientNet-B0 punya ~237 layers
# Untuk dataset kecil, unfreeze sedikit layer saja (misal mulai dari 200)
FINE_TUNE_AT = 200 # Semakin tinggi angka = semakin sedikit layer yang di-unfreeze

# TRAINING PHASE 1: Feature Extraction (Base Model Frozen)
LR_PHASE_1 = 1e-3 
EPOCHS_PHASE_1 = 20

# TRAINING PHASE 2: Fine-Tuning (Sebagian Base Model Unfrozen)
LR_PHASE_2 = 1e-5 # HARUS jauh lebih kecil untuk mencegah "catastrophic forgetting"
EPOCHS_PHASE_2 = 30

BATCH_SIZE = 32             # Jumlah gambar yang diproses sekali jalan.
OPTIMIZER_NAME = 'Adam'     # Pilihan: 'Adam', 'SGD', 'RMSprop'.

# LOSS FUNCTION
LOSS_TYPE = 'focal'  # 'focal' atau 'crossentropy'
LABEL_SMOOTHING = 0.1
FOCAL_ALPHA = 1.0
FOCAL_GAMMA = 2.0

# Nama file untuk menyimpan model terbaik
MODEL_SAVE_PATH = 'best_efficientnet_aflatoxin.keras'

# File log untuk mencatat semua eksperimen
LOG_FILE_PATH = 'experiment_log_efficientnet.csv'

# -----------------------------------------------------------------------------
# PRINT KONFIGURASI
# -----------------------------------------------------------------------------
print("="*60)
print("KONFIGURASI EKSPERIMEN")
print("="*60)
print(f"\n📐 ARSITEKTUR:")
print(f"   Dense Units: {DENSE_UNITS}")
print(f"   Dropout Rate: {DROPOUT_RATE}")
print(f"   Fine-tune dari layer: {FINE_TUNE_AT}")
print(f"\n📈 PHASE 1 (Feature Extraction):")
print(f"   Learning Rate: {LR_PHASE_1}")
print(f"   Epochs: {EPOCHS_PHASE_1}")
print(f"\n📈 PHASE 2 (Fine-Tuning):")
print(f"   Learning Rate: {LR_PHASE_2}")
print(f"   Epochs: {EPOCHS_PHASE_2}")
print(f"\n🎯 REGULARISASI:")
print(f"   Label Smoothing: {LABEL_SMOOTHING}")
print(f"   Loss Type: {LOSS_TYPE}")
if LOSS_TYPE == 'focal':
    print(f"   Focal Alpha: {FOCAL_ALPHA}")
    print(f"   Focal Gamma: {FOCAL_GAMMA}")

In [ ]:
# =============================================================================
# CELL 5: MEMBANGUN MODEL EFFICIENTNET-B0
# =============================================================================

def build_efficientnet_model():
    
    # 1. Load EfficientNet-B0 pre-trained
    base_model = EfficientNetB0(
        include_top=False,
        weights='imagenet',
        input_shape=(img_height, img_width, 3)
    )
    
    # 2. Freeze base model (Phase 1)
    base_model.trainable = False
    
    # 3. Bangun classifier head
    # Input layer
    inputs = Input(shape=(img_height, img_width, 3), name='input_layer')
    
    # Pass through EfficientNet
    # training=False: BatchNorm layers dalam mode inference
    x = base_model(inputs, training=False)
    
    # Global Average Pooling
    # Mengubah output (7, 7, 1280) menjadi (1280,)
    # Alternatif: GlobalMaxPooling2D atau Flatten
    x = GlobalAveragePooling2D(name='global_avg_pool')(x)
    
    # Dense layer dengan ReLU activation
    x = Dense(DENSE_UNITS, activation='relu', name='dense_1')(x)
    
    # Dropout untuk regularisasi
    x = Dropout(DROPOUT_RATE, name='dropout')(x)
    
    # Output layer dengan Softmax
    # 4 neuron untuk 4 kelas (1 PPB, 2 PPB, 3 PPB, 4 PPB)
    outputs = Dense(NUM_CLASSES, activation='softmax', name='output')(x)
    
    # -------------------------------------------------------------------------
    # 4. Buat model
    # -------------------------------------------------------------------------
    model = Model(inputs=inputs, outputs=outputs, name='EfficientNetB0_Aflatoxin')
    
    return model, base_model

# Bangun model
model, base_model = build_efficientnet_model()

# Tampilkan informasi model
print("="*60)
print("MODEL BERHASIL DIBANGUN")
print("="*60)
print(f"\nTotal parameters: {model.count_params():,}")
print(f"Trainable parameters: {sum([tf.keras.backend.count_params(w) for w in model.trainable_weights]):,}")
print(f"Non-trainable parameters: {sum([tf.keras.backend.count_params(w) for w in model.non_trainable_weights]):,}")
print(f"\nBase model layers: {len(base_model.layers)}")

In [ ]:
# CELL 7: COMPILE MODEL

# Pilih loss function
if LOSS_TYPE == 'focal':
    loss_fn = FocalLoss(alpha=FOCAL_ALPHA, gamma=FOCAL_GAMMA)
    loss_name = f"Focal Loss (α={FOCAL_ALPHA}, γ={FOCAL_GAMMA})"
else:
    loss_fn = tf.keras.losses.CategoricalCrossentropy(
        label_smoothing=LABEL_SMOOTHING
    )
    loss_name = f"Categorical Crossentropy (smoothing={LABEL_SMOOTHING})"

# Setup Optimizer Dinamis sesuai Konfigurasi
if OPTIMIZER_NAME.lower() == 'Adam':
    opt = Adam(learning_rate=LR_PHASE_1)
elif OPTIMIZER_NAME.lower() == 'sgd':
    opt = SGD(learning_rate=LR_PHASE_1)
elif OPTIMIZER_NAME.lower() == 'AdamW':
    opt = AdamW(learning_rate=LR_PHASE_1)
else:
    opt = RMSprop(learning_rate=LR_PHASE_1)
    
# Compile model
model.compile(
    optimizer=opt,
    loss=loss_fn,
    metrics=['accuracy']
)

print("Model compiled!")
print(f"  - Optimizer: {OPTIMIZER_NAME} (lr={LR_PHASE_1})")
print(f"  - Loss: {loss_name}")
print(f"  - Metrics: accuracy")
print(f"Model compiled successfully!")

In [ ]:
# =============================================================================
# CELL 8: SETUP CALLBACKS
# =============================================================================

callbacks = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        mode='max',
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-8,
        verbose=1
    ),
    ModelCheckpoint(
        MODEL_SAVE_PATH,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        verbose=1
    )
]

print("Callbacks configured:")
print("  ✓ EarlyStopping (patience=10, monitor=val_accuracy)")
print("  ✓ ReduceLROnPlateau (factor=0.5, patience=5)")
print(f"  ✓ ModelCheckpoint (save to: {MODEL_SAVE_PATH})")

In [ ]:
# =============================================================================
# CELL 9: PHASE 1 - FEATURE EXTRACTION
# =============================================================================
#
# TUJUAN: Melatih classifier head sementara base model dibekukan
#
# MENGAPA 2 PHASE?
# 1. Phase 1: Classifier head belajar menggunakan fitur dari EfficientNet
#    - Base model sudah punya fitur bagus dari ImageNet
#    - Kita hanya perlu melatih bagaimana menggunakan fitur tersebut
#
# 2. Phase 2: Fine-tune sebagian base model
#    - Setelah head sudah bagus, kita "tune" fitur untuk domain spesifik
#    - LR sangat kecil agar tidak merusak fitur yang sudah bagus
# =============================================================================

print("="*60)
print("PHASE 1: FEATURE EXTRACTION")
print("="*60)
print(f"\nBase model: FROZEN")
print(f"Training: Classifier head only")
print(f"Learning rate: {LR_PHASE_1}")
print(f"Epochs: {EPOCHS_PHASE_1}")
print("\nStarting training...\n")

history_phase1 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_PHASE_1,
    callbacks=callbacks,
    verbose=1
)

# Simpan hasil Phase 1
best_val_acc_p1 = max(history_phase1.history['val_accuracy'])
print(f"\n✓ Phase 1 Complete!")
print(f"✓ Best Validation Accuracy: {best_val_acc_p1:.4f}")

In [ ]:
# =============================================================================
# CELL 10: UNFREEZE BASE MODEL UNTUK FINE-TUNING
# =============================================================================
#
# STRATEGI FINE-TUNING:
#
# EfficientNet-B0 memiliki ~237 layers
# Layer awal: mendeteksi fitur umum (edges, textures)
# Layer akhir: mendeteksi fitur spesifik (patterns, objects)
#
# Untuk dataset kecil:
# - Freeze layer awal (fitur umum sudah bagus)
# - Unfreeze hanya beberapa layer akhir
# - FINE_TUNE_AT = 200 berarti layer 0-199 tetap frozen
# =============================================================================

print("="*60)
print("PREPARING PHASE 2: FINE-TUNING")
print("="*60)

# Unfreeze base model
base_model.trainable = True

# Freeze layer awal, unfreeze layer akhir
for layer in base_model.layers[:FINE_TUNE_AT]:
    layer.trainable = False

# Hitung statistik
total_layers = len(base_model.layers)
frozen_layers = FINE_TUNE_AT
trainable_layers = total_layers - FINE_TUNE_AT

print(f"\nBase model layers: {total_layers}")
print(f"Frozen layers: {frozen_layers} (0 to {FINE_TUNE_AT-1})")
print(f"Trainable layers: {trainable_layers} ({FINE_TUNE_AT} to {total_layers-1})")

if OPTIMIZER_NAME.lower() == 'Adam':
    opt_ft = Adam(learning_rate=LR_PHASE_2)
elif OPTIMIZER_NAME.lower() == 'sgd':
    opt_ft = SGD(learning_rate=LR_PHASE_2)
elif OPTIMIZER_NAME.lower() == 'AdamW':
    opt_ft = AdamW(learning_rate=LR_PHASE_2)
else:
    opt_ft = RMSprop(learning_rate=LR_PHASE_2)
    
# Re-compile dengan learning rate lebih kecil
# PENTING: Harus re-compile setelah mengubah trainable status
model.compile(
    optimizer=opt_ft,
    loss=loss_fn,
    metrics=['accuracy']
)

print(f"\n✓ Model re-compiled with LR={LR_PHASE_2}")
print(f"\nTrainable parameters now: {sum([tf.keras.backend.count_params(w) for w in model.trainable_weights]):,}")

In [ ]:
# =============================================================================
# CELL 11: PHASE 2 - FINE-TUNING
# =============================================================================

print("="*60)
print("PHASE 2: FINE-TUNING")
print("="*60)
print(f"\nBase model: PARTIALLY UNFROZEN (from layer {FINE_TUNE_AT})")
print(f"Learning rate: {LR_PHASE_2}")
print(f"Epochs: {EPOCHS_PHASE_2}")
print("\nStarting fine-tuning...\n")

# Lanjutkan dari epoch terakhir Phase 1
initial_epoch = len(history_phase1.history['loss'])

history_phase2 = model.fit(
    train_ds,
    validation_data=val_ds,
    initial_epoch=initial_epoch,
    epochs=initial_epoch + EPOCHS_PHASE_2,
    callbacks=callbacks,
    verbose=1
)

# Simpan hasil Phase 2
best_val_acc_p2 = max(history_phase2.history['val_accuracy'])
print(f"\n✓ Phase 2 Complete!")
print(f"✓ Best Validation Accuracy: {best_val_acc_p2:.4f}")

In [ ]:
print("Step 5: Evaluation on data set...")

# Load model terbaik
model = tf.keras.models.load_model('best_efficientnet_aflatoxin.keras')

# Evaluasi
test_loss, test_acc = model.evaluate(test_ds, verbose=1)
print(f"\n✓ Test Loss: {test_loss:.4f}")
print(f"✓ Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")

In [ ]:
# =============================================================================
# CELL 13: EVALUASI PADA TEST SET
# =============================================================================
# Prediksi
print("\nMelakukan prediksi...")

print(f"\nTEST ACCURACY: {test_accuracy*100:.2f}%")

In [ ]:

# =============================================================================
# CELL 12: VISUALISASI TRAINING HISTORY
# =============================================================================
y_pred_probs = model.predict(test_ds, verbose=1)
y_pred = np.argmax(y_pred_probs, axis=1)

# Ground truth
y_true_onehot = np.concatenate([y for x, y in test_ds], axis=0)
y_true = np.argmax(y_true_onehot, axis=1)

# Nama kelas untuk report
target_names = [f'Kelas {name})' for name in class_names]

# Hitung metrik
test_accuracy = accuracy_score(y_true, y_pred)


import matplotlib.pyplot as plt

# Gabungkan history dari kedua phase
acc = history_phase1.history['accuracy'] + history_phase2.history['accuracy']
val_acc = history_phase1.history['val_accuracy'] + history_phase2.history['val_accuracy']
loss = history_phase1.history['loss'] + history_phase2.history['loss']
val_loss = history_phase1.history['val_loss'] + history_phase2.history['val_loss']

epochs_range = range(1, len(acc) + 1)
phase1_end = len(history_phase1.history['accuracy'])

fig, axes = plt.subplots(1, 2, figsize=(8, 6))

# Plot Accuracy
axes[0].plot(epochs_range, acc, 'b-', label='Training Accuracy', linewidth=2)
axes[0].plot(epochs_range, val_acc, 'r-', label='Validation Accuracy', linewidth=2)
axes[0].axvline(x=phase1_end, color='green', linestyle='--', label='Phase 1 → 2')
axes[0].set_title('Model Accuracy', fontsize=14)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot Loss
axes[1].plot(epochs_range, loss, 'b-', label='Training Loss', linewidth=2)
axes[1].plot(epochs_range, val_loss, 'r-', label='Validation Loss', linewidth=2)
axes[1].axvline(x=phase1_end, color='green', linestyle='--', label='Phase 1 → 2')
axes[1].set_title('Model Loss', fontsize=14)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Siapkan Folder Penyimpanan Gambar
plot_folder = "visualizations" 
if not os.path.exists(plot_folder):
    os.makedirs(plot_folder) # Buat folder otomatis jika belum ada
# Buat Nama File Unik (Timestamp + Akurasi)
current_time_str = datetime.now().strftime("%Y%m%d_%H%M%S")
plot_filename = f"cm_{current_time_str}_Acc{test_accuracy*100:.1f}.png"
plotfilepath = os.path.join(plot_folder, plot_filename)

plt.savefig(plotfilepath, dpi=300, bbox_inches='tight')
plt.title(f'Visualisasi Hasil (Acc: {test_accuracy*100:.2f}%)')
plt.tight_layout()
plt.show()

In [ ]:
# =============================================================================
# CELL 14: CLASSIFICATION REPORT & CONFUSION MATRIX
# =============================================================================

import seaborn as sns

# Classification Report
print("\n" + "="*60)
print("CLASSIFICATION REPORT")
print("="*60)
report = classification_report(
    y_true, 
    y_pred, 
    target_names=target_names, 
    digits=4,
    output_dict=True
)
print(classification_report(y_true, y_pred, target_names=target_names, digits=4))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)

print("\nConfusion Matrix:")
print(cm)

# Siapkan Folder Penyimpanan Gambar
plot_folder = "history_plots_ENB0" 
if not os.path.exists(plot_folder):
    os.makedirs(plot_folder) # Buat folder otomatis jika belum ada
# Buat Nama File Unik (Timestamp + Akurasi)
current_time_str = datetime.now().strftime("%Y%m%d_%H%M%S")
plot_filename = f"cm_{current_time_str}_Acc{test_accuracy*100:.1f}.png"
plot_filepath = os.path.join(plot_folder, plot_filename)

# Plot Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(
    cm, 
    annot=True, 
    fmt='d', 
    cmap='Blues',
    xticklabels=target_names,
    yticklabels=target_names,
    annot_kws={'size': 14}
)
plt.title(f'Confusion Matrix - EfficientNet-B0\n(Test Accuracy: {test_accuracy*100:.2f}%)', fontsize=14)
plt.ylabel('Actual Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()

# Simpan gambar
plt.savefig(plot_filepath, dpi=300, bbox_inches='tight')
plt.show()

print(f"\n📊 Confusion matrix saved to: {plot_filepath}")

In [ ]:
# =============================================================================
# CELL 15: ANALISIS CONFIDENCE
# =============================================================================
#
# Confidence = seberapa "yakin" model dengan prediksinya
# High confidence + correct = bagus
# High confidence + wrong = berbahaya (overconfident)
# Low confidence = model ragu-ragu (mungkin data ambigu)
# =============================================================================

# Hitung confidence untuk setiap prediksi
confidence_scores = np.max(y_pred_probs, axis=1)

# Statistik
avg_confidence = np.mean(confidence_scores)
min_confidence = np.min(confidence_scores)
max_confidence = np.max(confidence_scores)
std_confidence = np.std(confidence_scores)

# Confidence untuk prediksi benar vs salah
correct_mask = (y_pred == y_true)
correct_confidence = np.mean(confidence_scores[correct_mask])
wrong_confidence = np.mean(confidence_scores[~correct_mask]) if (~correct_mask).sum() > 0 else 0

print("="*60)
print("ANALISIS CONFIDENCE")
print("="*60)
print(f"\nStatistik Overall:")
print(f"  - Average Confidence: {avg_confidence*100:.2f}%")
print(f"  - Min Confidence: {min_confidence*100:.2f}%")
print(f"  - Max Confidence: {max_confidence*100:.2f}%")
print(f"  - Std Confidence: {std_confidence*100:.2f}%")
print(f"\nConfidence by Prediction:")
print(f"  - Correct predictions: {correct_confidence*100:.2f}%")
print(f"  - Wrong predictions: {wrong_confidence*100:.2f}%")

# Interpretasi
if correct_confidence > wrong_confidence + 0.1:
    print("\n✓ Model lebih confident pada prediksi yang benar (bagus!)")
else:
    print("\n⚠️ Model cukup confident bahkan pada prediksi salah (hati-hati overconfidence)")

In [ ]:
# =============================================================================
# CELL 16: SIMPAN LOG EKSPERIMEN
# =============================================================================

# Buat dictionary dengan semua informasi eksperimen
log_data = {
    # Timestamp
    'Timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    
    # Arsitektur
    "Optimizer": OPTIMIZER_NAME,
    "Batch_Size": BATCH_SIZE,
    'Dense_Units': DENSE_UNITS,
    'Dropout': DROPOUT_RATE,
    'Fine_Tune_At': FINE_TUNE_AT,
    
    # Training params
    'LR_Phase_1': LR_PHASE_1,
    'Epochs_P1': EPOCHS_PHASE_1,
    'LR_Phase_2': LR_PHASE_2,
    'Epochs_P2': EPOCHS_PHASE_2,
    
    # Regularisasi
    'Loss_Type': LOSS_TYPE,
    'Label_Smoothing': LABEL_SMOOTHING if LOSS_TYPE == 'crossentropy' else 'N/A',
    'Focal_Gamma': FOCAL_GAMMA if LOSS_TYPE == 'focal' else 'N/A',
    
    # Hasil
    'Accuracy': round(test_accuracy, 4),
    'Avg_Confidence': round(avg_confidence, 4),
    'Confusion_Matrix': str(cm.tolist()),
}

# Tambahkan metrik per kelas
for name in target_names:
    log_data[f'{name}_Precision'] = round(report[name]['precision'], 4)
    log_data[f'{name}_Recall'] = round(report[name]['recall'], 4)
    log_data[f'{name}_F1'] = round(report[name]['f1-score'], 4)

# Simpan ke CSV
df_log = pd.DataFrame([log_data])

if os.path.exists(LOG_FILE_PATH):
    df_log.to_csv(LOG_FILE_PATH, mode='a', header=False, index=False)
    print(f"✓ Log ditambahkan ke: {LOG_FILE_PATH}")
else:
    df_log.to_csv(LOG_FILE_PATH, index=False)
    print(f"✓ File log baru dibuat: {LOG_FILE_PATH}")

print("\n" + "="*60)
print("EKSPERIMEN SELESAI")
print("="*60)
print(f"\n📁 Model tersimpan di: {MODEL_SAVE_PATH}")
print(f"📁 Log tersimpan di: {LOG_FILE_PATH}")
print(f"\n🎯 Test Accuracy: {test_accuracy*100:.2f}%")

---

## 🔄 Langkah Selanjutnya

### Jika akurasi belum memuaskan, coba:

1. **Ubah FINE_TUNE_AT**: 
   - Lebih kecil (misal 150) = lebih banyak layer di-train = lebih adaptif tapi risiko overfitting
   - Lebih besar (misal 220) = lebih sedikit layer = lebih stable

2. **Ubah DENSE_UNITS**:
   - Coba [128] atau [512]
   
3. **Ubah DROPOUT_RATE**:
   - Naikan ke 0.6 jika overfitting
   - Turunkan ke 0.3 jika underfitting

4. **Coba loss function lain**:
   - Ubah LOSS_TYPE ke 'crossentropy' dengan LABEL_SMOOTHING=0.2

5. **Coba arsitektur lain**:
   - EfficientNet-B1 (sedikit lebih besar)
   - DenseNet-121
   - MobileNetV2

---